#### Step 1: Rename files and make sure every chip file has an associated input file
All the data was manually reviewed and renamed for human readability. Since there was a single input sample seqenced that would serve as an input for several chipseq files, the input was duplicated such that each chipseq file has an associated input file. The data was then transfered to O2 for processing. <br>

```
srun --pty -p interactive --mem 80G -t 0-06:00 /bin/bash
gunzip *.gz
mkdir reference
wget ftp://ftp.ccb.jhu.edu/pub/data/bowtie2_indexes/hg19.zip
unzip hg19.zip
```

#### Step 2: 

```
vim run_1.sh
```

```
#!/bin/sh
#SBATCH -p short
#SBATCH -J bowtie2_O2
#SBATCH -o run.o
#SBATCH -e run.e
#SBATCH -t 0-12:00                                                                                                    #SBATCH --cpus-per-task=1
#SBATCH --mem=8000
#SBATCH --mail-type=ALL
#SBATCH --mail-user=ajit_nirmal@hms.harvard.edu

module load gcc/6.2.0 bowtie2/2.3.4.3

/n/app/bowtie2/2.3.4.3/bin/bowtie2 -x hg19 -1 /n/scratch2/ajit/gw_chip/fastq/GW_K27Ac_2_FRRC190725836-1b_H3KNFBBXX_L7_1.fq.gz -2 /n/scratch2/ajit/gw_chip/fastq/GW_K27Ac_2_FRRC190725836-1b_H3KNFBBXX_L7_2.fq.gz -S GW_K27Ac_2.sam
```

#### Step 3:
The above generated SAM files were converted into BAM files and sorted using samtools. <br>

```
#!/bin/sh
#SBATCH -p short
#SBATCH -J bowtie2_O2_samtobam
#SBATCH -o run.osamtobam
#SBATCH -e run.esamtobam
#SBATCH -t 0-4:00
#SBATCH --cpus-per-task=1
#SBATCH --mem=8000
#SBATCH --mail-type=ALL
#SBATCH --mail-user=ajit_nirmal@hms.harvard.edu

module load gcc/6.2.0 samtools/1.9

samtools view -S -b GW_K27Ac_2.sam > GW_K27Ac_2.bam
samtools sort GW_K27Ac_2.bam -o sorted_GW_K27Ac_2.bam
```

##### Using BAM files: MACS will use the real fragments inferred from alignment results for reads pileup.

```
#!/bin/sh
#SBATCH -p short
#SBATCH -J bowtie2_O2_J
#SBATCH -o run.om
#SBATCH -e run.em
#SBATCH -t 0-03:00
#SBATCH --cpus-per-task=1
#SBATCH --mem=8000
#SBATCH --mail-type=ALL
#SBATCH --mail-user=ajit_nirmal@hms.harvard.edu

module load gcc/6.2.0 python/2.7.12 macs2/2.1.1.20160309

macs2 callpeak -t sorted_GW_TP63.bam -f BAMPE -g hs -n JPE -B -q 0.05
```
